In [ ]:
import os
from pathlib import Path
from ultralytics.models import YOLO
from roboflow import Roboflow
from dotenv import load_dotenv
import torch
from multiprocessing import Pool
from decouple import config

In [ ]:
rf = Roboflow(api_key=config("ROBOFLOW_KEY", default=""))
project = rf.workspace("udla").project("football-players-detection-v2-shnaf")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


In [10]:
data_yaml = dataset.location + "/data.yaml"
print(f"Dataset descargado en: {dataset.location}")

model = YOLO("yolov8m.pt")

Dataset descargado en: c:\Users\Usuario\Downloads\train yolo\football-players-detection-v2-1


In [22]:
output_path = "trained_exports"
# Recomendado: usa este train() actualizado
import torch
use_cuda = torch.cuda.is_available()
device = "cuda" if use_cuda else "cpu"
if use_cuda:
    batch = 16
else:
    batch = 4

def train(model: YOLO):
    # Detectar dispositivo y ajustar batch/lr automáticamente
    
    
    lr0 = 0.01 if batch >= 16 else 0.001

    model.train(
        data=data_yaml,
        epochs=60,
        patience=20,
        imgsz=640,              # coincidencia con export Roboflow
        device=device,
        batch=batch,            # batch fijo adaptado
        workers=8 if use_cuda else 0,
        optimizer="SGD",
        lr0=lr0,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.0005,
        cos_lr=True,
        warmup_epochs=3,
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        degrees=0.5,
        translate=0.1,
        scale=0.3,
        shear=0.0,
        perspective=0.0005,
        flipud=0.1,
        fliplr=0.5,
        mosaic=0.5,             # reducir intensidad de mosaic para dataset pequeño
        mixup=0.0,              # mixup puede confundir para objetos pequeños (balón)
        rect=True,              # mantener relación de aspecto / mejor correspondencia bbox
        cache=True,             # cache para acelerar y detectar I/O/format issues
        save=True,
        amp=True,
        plots=True,
        verbose=True
    )



def predict(model: YOLO):
    model.val()
    model.predict(source="/content/football-players-detection-v2-1/test", conf=0.25, device="cuda")

def export(model: YOLO):
    path = model.export(
    format=[
        "onnx",
        "torchscript",
        "coreml",
        "openvino"
    ],
    imgsz=960,
    save_dir=output_path
)
    print(path)

In [21]:
import torch
from pathlib import Path
from ultralytics import YOLO
import hashlib

# Ruta al modelo
MODEL_NAME = "yolov8m.pt"
EXPECTED_HASH = "a6e6aaed220d7830f5b30206d6512b9d"  # SHA256 de yolov8m.pt oficial

def sha256_hash(file_path):
    """Calcula el hash SHA256 de un archivo."""
    hash_sha256 = hashlib.sha256()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_sha256.update(chunk)
    return hash_sha256.hexdigest()

def verify_and_download_yolov8m():
    model_path = Path(MODEL_NAME)

    if model_path.exists():
        print(f"⚠️ Archivo encontrado: {model_path}")
        print("🔍 Verificando integridad...")
        if sha256_hash(model_path) == EXPECTED_HASH:
            print("✅ El archivo es válido. Usando modelo local.")
            return str(model_path.resolve())
        else:
            print("⚠️ El archivo está corrupto o no es yolov8m.pt. Eliminando...")
            model_path.unlink()

    print("⬇️ Descargando yolov8m.pt oficial...")
    model = YOLO(MODEL_NAME)  # Esto descarga solo si no existe o está corrupto
    return str(Path(MODEL_NAME).resolve())

# Uso
yolo_model_path = verify_and_download_yolov8m()
print(f"✅ Modelo listo en: {yolo_model_path}")

⚠️ Archivo encontrado: yolov8m.pt
🔍 Verificando integridad...
⚠️ El archivo está corrupto o no es yolov8m.pt. Eliminando...
⬇️ Descargando yolov8m.pt oficial...
✅ Modelo listo en: /content/yolov8m.pt


In [23]:
train(model)

New https://pypi.org/project/ultralytics/8.3.235 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.234  Python-3.13.5 torch-2.9.1+cpu CPU (AMD Ryzen 7 8700F 8-Core Processor)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=c:\Users\Usuario\Downloads\train yolo\football-players-detection-v2-1/data.yaml, degrees=0.5, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=0.5, mult

In [ ]:
export(YOLO("./"))

Ultralytics 8.3.195 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
YOLO11x summary (fused): 190 layers, 56,829,334 parameters, 0 gradients, 194.4 GFLOPs

PyTorch: starting from '/content/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (109.1 MB)

TorchScript: starting export with torch 2.8.0+cu126...
TorchScript: export success ✅ 35.9s, saved as '/content/runs/detect/train/weights/best.torchscript' (217.8 MB)

Export complete (42.1s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=/content/runs/detect/train/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=/content/runs/detect/train/weights/best.torchscript imgsz=640 data=/content/football-players-detection-v2-1/data.yaml  
Visualize:       https://netron.app
Ultralytics 8.3.195 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
YOLO11x summary (fused): 190 layer

In [ ]:
# Compress the output directory
!zip -r /content/output.zip /content/runs/detect/train

# Provide a link to download the zip file
from google.colab import files
files.download('/content/output.zip')

updating: content/runs/detect/train/ (stored 0%)
updating: content/runs/detect/train/args.yaml (deflated 52%)
updating: content/runs/detect/train/BoxF1_curve.png (deflated 13%)
updating: content/runs/detect/train/val_batch0_pred.jpg (deflated 6%)
updating: content/runs/detect/train/val_batch2_labels.jpg (deflated 6%)
updating: content/runs/detect/train/labels.jpg (deflated 31%)
updating: content/runs/detect/train/confusion_matrix_normalized.png (deflated 32%)
updating: content/runs/detect/train/confusion_matrix.png (deflated 34%)
updating: content/runs/detect/train/train_batch2.jpg (deflated 1%)
updating: content/runs/detect/train/train_batch1.jpg (deflated 2%)
updating: content/runs/detect/train/BoxP_curve.png (deflated 18%)
updating: content/runs/detect/train/train_batch31360.jpg (deflated 6%)
updating: content/runs/detect/train/BoxPR_curve.png (deflated 17%)
updating: content/runs/detect/train/val_batch2_pred.jpg (deflated 6%)
updating: content/runs/detect/train/train_batch31361.jpg

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>